In [4]:
import numpy as np
import pandas as pd
from scipy.fft import fft, ifft
from scipy.interpolate import interp1d
from scipy.stats import entropy, iqr, kurtosis, mode, skew
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import (Activation, BatchNormalization, Bidirectional,
                                     Conv1D, Conv2D, Dense, Dropout,
                                     Flatten, GlobalAveragePooling1D, LSTM,
                                     MaxPool1D, MaxPooling2D, Reshape,
                                     TimeDistributed)
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import os
import pickle
from pandas import read_csv, unique
from scipy.signal import argrelextrema, butter, find_peaks, lfilter, lfilter_zi
from tensorflow import keras
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_accuracy
from kerastuner import RandomSearch, Hyperband
import kerastuner


In [10]:
X_respeck = pickle.load(open("X_respeck_resp_other", "rb"))
y_respeck = pickle.load(open("y_respeck_resp_other", "rb"))
y_respeck_phys = [[x[0].split('|')[0]] for x in y_respeck]
y_respeck = [[x[0].split('|')[1]] for x in y_respeck] # get only subactivities
print(X_respeck.shape)
y_respeck_task2 = []
for i in range(len(y_respeck)):
    if y_respeck[i][0] in ['normal', 'hyperventilating', 'coughing']:
        y_respeck_task2.append(y_respeck[i][0])
    else:
        y_respeck_task2.append('other')     
y_respeck_task2 = np.array(y_respeck_task2).reshape(-1,1)
enc = OneHotEncoder(handle_unknown='ignore')

enc_fitted = enc.fit(y_respeck_task2)
enc_fitted.categories_
y_respeck_task2 = enc.transform(y_respeck_task2).toarray()
print(enc_fitted.categories_)# One Hot is inexed as this array prints!!!!!
y_respeck_task2 = np.array(y_respeck_task2)


X_respeck_task3 = []
y_respeck_task3 = []
for i in range(len(y_respeck)):
    if not y_respeck[i][0] in ['normal', 'hyperventilating', 'coughing']:
        X_respeck_task3.append(X_respeck[i])
        y_respeck_task3.append(y_respeck[i][0])
y_respeck_task3 = np.array(y_respeck_task3).reshape(-1,1)
enc_fitted = enc.fit(y_respeck_task3)
enc_fitted.categories_
y_respeck_task3 = enc.transform(y_respeck_task3).toarray()
print(enc_fitted.categories_)# One Hot is inexed as this array prints!!!!!

(49518, 50, 10)
[array(['coughing', 'hyperventilating', 'normal', 'other'], dtype='<U16')]
[array(['eating', 'laughing', 'singing', 'talking'], dtype='<U8')]


In [6]:
X_train, X_temp, y_train, y_temp = train_test_split(X_respeck, y_respeck_task2, test_size=0.2, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [7]:
def build_model(hp):
    model = Sequential()
    model.add(LSTM(units=hp.Int('lstm_units1', min_value=16, max_value=128, step=16), return_sequences=True, input_shape=X_train[0].shape, activation='relu'))
    model.add(LSTM(units=hp.Int('lstm_units2', min_value=16, max_value=128, step=16), return_sequences=True, activation='relu'))
    model.add(Conv1D(filters=hp.Int('conv1d_filters1', min_value=16, max_value=128, step=16), kernel_size=hp.Int('conv1d_kernel_size1', min_value=2, max_value=5), activation='relu', strides=2))
    model.add(MaxPool1D(pool_size=hp.Int('maxpool1d_pool_size', min_value=2, max_value=5), padding='same'))
    model.add(Conv1D(filters=hp.Int('conv1d_filters2', min_value=16, max_value=128, step=16), kernel_size=hp.Int('conv1d_kernel_size2', min_value=2, max_value=5), activation='relu', strides=1))
    model.add(GlobalAveragePooling1D())
    model.add(BatchNormalization(epsilon=1e-06))
    model.add(Dense(y_train.shape[1], activation='softmax'))

    # Compile the model with tunable hyperparameters
    model.compile(
        optimizer=Adam(learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='log')),
        loss=categorical_crossentropy,
        metrics=['categorical_accuracy']
    )
    
    return model

In [8]:
from kerastuner.tuners import BayesianOptimization

# tuner = Hyperband(
#     build_model,
#     factor=3,  # Reduction factor for the number of models in each bracket
#     max_epochs=20,  # Maximum number of training epochs for each model
#     objective='categorical_accuracy',  # The metric to optimize
#     directory='tuner_dir',  # Directory to store results
#     project_name='cnn_lstm_task_no_pre', # Name of the tuning project
# )

tuner = BayesianOptimization(
    build_model,
    objective='categorical_accuracy',  # The metric to optimize
    max_trials=10,  # Number of trials
    # num_initial_points=5,  # Number of initial random trials
    directory='tuner_dir',  # Directory to store results
    project_name='task3_task2_sub'  # Name of the tuning project
)



2023-11-03 11:21:49.202349: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-11-03 11:21:49.202396: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-11-03 11:21:49.202423: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (dan-HP-ENVY-x360-Convertible-13-ay0xxx): /proc/driver/nvidia/version does not exist
2023-11-03 11:21:49.202850: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
tuner.search(X_train, y_train, validation_data=(X_val, y_val), batch_size=64, shuffle=False)


Search: Running Trial #1

Hyperparameter      |Value     |Best Value So Far   
lstm_units1         |96        |?                   
lstm_units2         |128       |?                   
conv1d_filters1     |16        |?                   
conv1d_kernel_size1 |3         |?                   
maxpool1d_pool_size |3         |?                   
conv1d_filters2     |64        |?                   
conv1d_kernel_size2 |4         |?                   
learning_rate       |0.0006204649496530797|?                   

522/619 [========================>.....] - ETA: 15s - loss: 1.1009 - categorical_accuracy: 0.5191

KeyboardInterrupt: 

In [9]:
task1 = keras.models.load_model('Trained_Task1.keras')

2023-11-03 10:44:00.787691: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-11-03 10:44:00.787857: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-11-03 10:44:00.787902: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (dan-HP-ENVY-x360-Convertible-13-ay0xxx): /proc/driver/nvidia/version does not exist
2023-11-03 10:44:00.788322: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [12]:
def predict_task3(X):
    physical_action = task1.predict(X)
    physical_action = np.argmax(physical_action, axis=1)
    prediction_labels = ['ascending stairs', 'descending stairs', 'lying down back',
        'lying down on left', 'lying down on stomach', 'lying down right',
        'miscellaneous movements', 'normal walking', 'running',
        'shuffle walking', 'sitting', 'standing']
    physical_prediction = [prediction_labels[i] for i in physical_action]
    subtype_prediction = task3.predict(X)
    subtype_prediction = np.argmax(subtype_prediction, axis=1)
    
predict_task3(X_test)

180/180 [==============================] - 5s 26ms/step
lying down on left
